In [ ]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import os
import progressbar
from konlpy.tag import Okt
# from selenium import webdriver
from bs4 import BeautifulSoup
# from selenium.webdriver.common.by import By
# from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.chrome.service import Service
# from webdriver_manager.chrome import ChromeDriverManager
import time

In [ ]:
main_path = '/content/drive/MyDrive/Project/LiveCommerse/'
# main_path = '/Users/jang/Library/CloudStorage/GoogleDrive-gkfua00@gmail.com/내 드라이브/Project/LiveCommerse/'
# main_path = 'D:\Google Drive\내 드라이브\Project\LiveCommerse//'

In [ ]:
df = pd.read_csv(main_path + 'Total_DataFrame.csv')
df = df[df['Tour/Experience'] == 1]
df.Num_Video = df.Num_Video.astype(int)
df = df[(df['Food']==0) & (df['Beauty']==0) & (df['Life']==0) & (df['Fashion']==0) & (df['Kids']==0) & (df['Tech']==0) & (df['Hobby/Leisure']==0) & (df['Culture']==0)]
df.reset_index(drop=True, inplace=True)

C:\Users\gkfua\AppData\Local\Temp\ipykernel_37912\1212355570.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(main_path + 'Total_DataFrame.csv')


In [ ]:
sentiword = main_path + 'KnuSentiLex-master/KnuSentiLex-master/SentiWord_Dict.txt'
dictionary = pd.read_csv(sentiword, sep='\t', header=None, names=['word', 'score'])
okt = Okt()

def analyze_sentiment(text):
    words = okt.pos(text, stem=True, norm=True)
    word_scores = []
    for word, pos in words:
        if pos == 'Adjective' or pos == 'Verb' or pos =='Noun':
            try:
                score = dictionary.loc[dictionary['word'] == word]['score'].values[0]
                word_scores.append(score)
            except:
                continue

    if len(word_scores) > 0:
        avg_score = sum(word_scores) / len(word_scores)
    else:
        avg_score = 0.0

    if avg_score > 0:
        sentiment = 'Positive'
    elif avg_score < 0:
        sentiment = 'Negative'
    else:
        sentiment = 'Neutral'

    return sentiment, avg_score

In [ ]:
Path_1 = '/content/drive/MyDrive/Project/LiveCommerse/FUCKUGOOGLE/nonTour_df'
Path_2 = '/content/drive/MyDrive/Project/LiveCommerse/FUCKUGOOGLE/Tour_only_df'

In [ ]:
Mall_super_list = os.listdir(Path_1)
Mall_tour_list = os.listdir(Path_2)

Mall_super_list.remove('Complete_list.pkl')
Mall_super_list.remove('Check_list.pkl')
Mall_tour_list.remove('Complete_list.pkl')
Mall_tour_list.remove('Check_list.pkl')

In [ ]:
Mall_tour_list[50:]

In [ ]:
for i in Mall_tour_list[20:33]:
  Video_Path = Path_2 + f'/{i}'
  video_list = os.listdir(Video_Path)

  for j in video_list:
    if 'csv' not in j:
      video_list.remove(j)
      print(f'remove : {j}')

  for k in video_list:
    if len(k) > 17:
      video_list.remove(k)
      print(f'remove : {k}')

  for video in video_list:
    try:
      video_path = Video_Path + f'/{video}'
      temp_csv = pd.read_csv(video_path)
      temp_csv['Sentiment'] = 0
      temp_csv['Score'] = 0
      temp_csv['chat'] = temp_csv['chat'].astype(str)
      temp_list = []
      okt = Okt()

      for j in range(len(temp_csv)):
        temp_csv['Sentiment'][j] = analyze_sentiment(temp_csv['chat'][j])[0]
        temp_csv['Score'][j] = analyze_sentiment(temp_csv['chat'][j])[1]
    except:
      print('error')
    temp_csv.to_csv(video_path)